In [2]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import time, os
from dotenv import load_dotenv
load_dotenv()

/Users/joonhokim/Library/Caches/pypoetry/virtualenvs/vectordb-f7pNHS7S-py3.11/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


True

In [3]:
PINECONE_API_KEY=os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key=PINECONE_API_KEY)

In [30]:
#pc.delete_index("my-agent-index")


### Generate vectors

In [5]:
# Define a sample dataset where each item has a unique ID and piece of text
data = [
    {"id": "vec1", "text": "Apple is a popular fruit known for its sweetness and crisp texture."},
    {"id": "vec2", "text": "The tech company Apple is known for its innovative products like the iPhone."},
    {"id": "vec3", "text": "Many people enjoy eating apples as a healthy snack."},
    {"id": "vec4", "text": "Apple Inc. has revolutionized the tech industry with its sleek designs and user-friendly interfaces."},
    {"id": "vec5", "text": "An apple a day keeps the doctor away, as the saying goes."},
    {"id": "vec6", "text": "Apple Computer Company was founded on April 1, 1976, by Steve Jobs, Steve Wozniak, and Ronald Wayne as a partnership."}
]

In [6]:
# Convert the text into numerical vectors that Pinecone can index
embeddings = pc.inference.embed(
    model="multilingual-e5-large",
    inputs=[d['text'] for d in data], # 데이터에서 텍스트리스트
    parameters={"input_type": "passage", "truncate": "END"} # 임베딩 모델 파라미터, 입력 데이터의 유형을 "passage"로 설정하고, 텍스트가 길 경우 끝에서 잘리도록 지정
)

In [7]:
print(embeddings)


EmbeddingsList(
  model='multilingual-e5-large',
  data=[
    {'values': [0.04913330078125, -0.01306915283203125, ..., -0.0196990966796875, -0.0110321044921875]},
    {'values': [0.032470703125, -0.027923583984375, ..., -0.020050048828125, -0.02099609375]},
    ... (2 more embeddings) ...,
    {'values': [0.0312347412109375, -0.0186309814453125, ..., -0.02996826171875, -0.033111572265625]},
    {'values': [0.039520263671875, -0.00997161865234375, ..., 0.0011930465698242188, -0.042755126953125]}
  ],
  usage={'total_tokens': 130}
)


### Create index

In Pinecone, you store data in an index.

In [17]:
existing_indexes = pc.list_indexes()
for index in existing_indexes:
    print(index)

{'deletion_protection': 'disabled',
 'dimension': 1024,
 'host': 'my-agent-index-1ps33nm.svc.aped-4627-b74a.pinecone.io',
 'metric': 'dotproduct',
 'name': 'my-agent-index',
 'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
 'status': {'ready': True, 'state': 'Ready'}}


In [19]:
index_name = "my-agent-index"


pc.create_index(
    name=index_name, 
    dimension=1024, 
    metric="dotproduct", 
    spec=ServerlessSpec(cloud="aws", region="us-east-1")
)
        
# Wait for the index to be ready
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '29407e81fb1f71dde668112a994bbff6', 'Date': 'Sun, 12 Jan 2025 11:02:40 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


### Describe index

In [30]:
pc.describe_index(index_name)

{'deletion_protection': 'disabled',
 'dimension': 1024,
 'host': 'my-agent-index-1ps33nm.svc.aped-4627-b74a.pinecone.io',
 'metric': 'dotproduct',
 'name': 'my-agent-index',
 'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
 'status': {'ready': True, 'state': 'Ready'}}

### Upsert vectors

In [22]:
index = pc.Index(index_name)

records = []
for d, e in zip(data, embeddings):
    records.append({
        "id": d['id'],
        "values": e['values'],
        "metadata": {'text': d['text']}
    })

index.upsert(
    vectors=records,
    namespace="example-namespace"
)

upserted_count: 6

In [24]:
print(index.describe_index_stats())

{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {'example-namespace': {'vector_count': 6}},
 'total_vector_count': 6}


### Search the index

In [29]:
# Define your query
query = "Tell me about the tech company known as Apple."


query_embedding = pc.inference.embed(
    model="multilingual-e5-large",
    inputs=[query],
    parameters={"input_type": "query"}
)

results = index.query(
    namespace="example-namespace",
    vector=query_embedding[0].values,
    top_k=3,
    include_values=False,
    include_metadata=True
)
print(results)

{'matches': [{'id': 'vec1',
              'metadata': {'text': 'Apple is a popular fruit known for its '
                                   'sweetness and crisp texture.'},
              'score': 0.7083072,
              'sparse_values': {'indices': [], 'values': []},
              'values': []},
             {'id': 'vec4',
              'metadata': {'text': 'Apple Inc. has revolutionized the tech '
                                   'industry with its sleek designs and '
                                   'user-friendly interfaces.'},
              'score': 0.69715697,
              'sparse_values': {'indices': [], 'values': []},
              'values': []},
             {'id': 'vec2',
              'metadata': {'text': 'The tech company Apple is known for its '
                                   'innovative products like the iPhone.'},
              'score': 0.6890224,
              'sparse_values': {'indices': [], 'values': []},
              'values': []}],
 'namespace': 'example-n